In [1]:
# in reference to https://github.com/DataTalksClub/llm-zoomcamp/blob/main/03-vector-search/demo_es.ipynb

# Step 1: Prepare documents


In [2]:
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[1]

{'text': 'GitHub - See DE-zoomcamp  prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

# Step 2: Create Embeddings using Pretrained Models


In [4]:
# This is a new library compared to the previous modules. 
# Please perform "pip install sentence_transformers==2.7.0"
from sentence_transformers import SentenceTransformer

# if you get an error do the following:
# 1. Uninstall numpy 
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work
model = SentenceTransformer("all-mpnet-base-v2")

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
len(model.encode("This is a simple sentence"))

768

In [6]:
documents[1]


{'text': 'GitHub - See DE-zoomcamp  prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
#created the dense vector using the pre-trained model
from tqdm import tqdm
operations = []
for doc in tqdm(documents):
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

  0%|          | 0/1031 [00:00<?, ?it/s]

100%|██████████| 1031/1031 [05:45<00:00,  2.99it/s]


Step 3: Setup ElasticSearch connection


In [9]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': 'd51a21c0aaf5', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'ngpXGGzuQASTTcf_7B4PvQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

# Step 4: Create Mappings and Index


In [8]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [10]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

# Step 5: Add documents into index


In [11]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

# Step 6: Create end user query


In [12]:
search_term = "windows or mac?"
vector_search_term = model.encode(search_term)
vector_search_term

array([-6.35387972e-02,  2.65321992e-02,  9.98955406e-03, -1.36715425e-02,
       -1.28370505e-02,  9.20389593e-03, -8.48796801e-04, -3.05588599e-02,
       -2.46248720e-03,  1.72226559e-02,  4.31264937e-02,  8.57972726e-03,
       -1.93418264e-02,  1.04178376e-02, -4.21266034e-02,  1.33134676e-02,
        9.56985503e-02, -3.55436690e-02,  1.66664664e-02, -1.86404586e-02,
       -5.06161666e-03,  1.73184052e-02, -1.59354173e-02, -2.41836086e-02,
       -1.45203508e-02, -3.99563722e-02, -7.25806206e-02,  2.68063098e-02,
       -5.12384400e-02,  5.11277318e-02, -3.41738723e-02,  1.96610317e-02,
        2.75925919e-02,  4.62351888e-02,  1.54686825e-06, -1.40663469e-02,
        1.84369516e-02,  4.19801148e-03,  5.42885251e-03,  1.89430565e-02,
        3.05294357e-02,  4.25610319e-02,  1.21717236e-03, -8.74174014e-03,
        1.01104900e-02, -2.74074133e-02, -2.32483838e-02,  3.35226543e-02,
       -2.28761397e-02,  2.62946319e-02, -2.22461130e-02,  5.40548330e-03,
       -1.50831891e-02, -

In [12]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}

NameError: name 'vector_search_term' is not defined

In [14]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'GsYNOZIBAeqSt1rO5-OK',
  '_score': 0.7147919,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'P8YOOZIBAeqSt1rOQeY3',
  '_score': 0.61072737,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares a

# Step 7: Perform Keyword search with Semantic Search (Hybrid/Advanced Search)


In [15]:
# Note: I made a minor modification to the query shown in the notebook here
# (compare to the one shown in the video)
# Included "knn" in the search query (to perform a semantic search) along with the filter  
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [16]:
response = es_client.search(
    index=index_name,
    query={
        "match": {"section": "General course-related questions"},
    },
    knn=knn_query,
    size=5
)

In [17]:
response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'GsYNOZIBAeqSt1rO5-OK',
  '_score': 11.635637,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.026965461671352386,
    -0.000626126304268837,
    -0.01662949100136757,
    0.05285150930285454,
    0.05476527288556099,
    -0.03133990615606308,
    0.029942581430077553,
    -0.04808562621474266,
    0.04467551037669182,
    0.005839474033564329,
    0.016233040019869804,
    0.012001154012978077,
    -0.031222281977534294,
    0.016600528731942177,
    -0.04886901378631592,
    -0.06496307998895645,
    0.046434223651885986,
    -0.009297756478190422,
    -0.0642528235912323,
    -0.01373267825692892,
    -0.015976183116436005,
    0.008629541844129562,
    -0.02447899058461